## Import of libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

## Import of CSV data

In [2]:
hashed_df = pd.read_csv('hashed_feature.csv')
tabular_df = pd.read_csv('tabular_data.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
tabular_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,1,110.0,55.0,0.432017,0.0,176.78,0.0,0.323712,NaN,...,0.00,0,0.0,0.0,55.0,2.0,0.526552,145.0,133.28,0.0
1,0,2,110.0,110.0,0.397517,0.0,315.42,0.0,0.316798,NaN,...,0.00,0,0.0,0.0,110.0,1.0,0.481063,130.0,229.97,0.0
2,0,3,110.0,55.0,0.359440,0.0,354.55,0.0,0.339188,NaN,...,0.07,0,0.0,0.0,55.0,1.0,0.509598,180.0,231.78,0.0


## Simplest solution to deal with NaN 

In [3]:
tabular_df = tabular_df.fillna(axis='columns', method='bfill')
tabular_df.head(3)

,id,period,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,0,1,110,55,0.432017,0,176.78,0,0.323712,147,...,0,0,0,0,55,2,0.526552,145,133.28,0
1,0,2,110,110,0.397517,0,315.42,0,0.316798,135,...,0,0,0,0,110,1,0.481063,130,229.97,0
2,0,3,110,55,0.35944,0,354.55,0,0.339188,181,...,0.07,0,0,0,55,1,0.509598,180,231.78,0


## Encode hashes

In [ ]:
def hashCounter():
    i = 1
    while key < (key + 1):
        i += 1

### Create hash dictionary to encode them in the tabular data

In [4]:

# group/filter so unique hash names and create a 'dictionary' dataframe
unique_hashes_df = pd.DataFrame(hashed_df.feature_50.unique(), columns = ['feature_50'])

hashes_dict_df = unique_hashes_df
# create a new 'id' column, set 'feature_50' as a key and 'hash_id' as a value to for easier future replacement/matching
# drop index name and create a new 'id' column based on indexes

hashes_dict_df.reset_index(level=0, inplace=True)

hashes_dict_df = hashes_dict_df.set_index('feature_50')

hashes_dict_df.index.name = None
hashes_dict_df = hashes_dict_df.rename(columns = {'index': 'id'})


### Convert dictionary dataframe into actual python dictionary while also transposing it 
### It is needed for mapping them to replace hash codes with integer values - that can be used for model algorithm to evaluate them, it shouldn't be  possible with just hashes as strings (from what I understand)

In [5]:
hash_codes = hashes_dict_df.T.to_dict('list')

### Encode (map) main hash table with integer id's from above dictionary

In [6]:
hashed_df['feature_50'] = hashed_df['feature_50'].map(hash_codes)

# fucntion that is applied to convert 'feature_50' lists column into integers
def convert_to_int(x):
    return x[0]

hashed_df['feature_50'] = hashed_df['feature_50'].apply(convert_to_int)
print(hashed_df.head(3))

   id  feature_50
0   0           0
1   0           1
2   0           2


In [ ]:
hashed_df_ = hashes_dict_df.T.to_dict('list')

## Restructure dataframe to put 12 periods in 1 data cell

## Find mean values of 12 periods

In [8]:
tabular_df_means = tabular_df.groupby([tabular_df['id']]).mean()
tabular_df_means.reset_index(level=0, inplace=True)
# fill NA again to make the concept work
tabular_df_means = tabular_df_means.fillna(axis='columns', method='bfill')
tabular_df_means

DataError: No numeric types to aggregate

## Buildign logistic regression model without hashes

In [ ]:
df_train = pd.merge(tabular_df_means, train_df, on='id')
y = df_train.target
X = df_train.drop(['period', 'id', 'target'], axis=1).fillna(method='bfill')

model = LogisticRegression(random_state=0, solver='newton-cg', multi_class='multinomial').fit(X, y)
model.score(X, y)

## Predicting/testing

In [ ]:
test_df_n = pd.merge(tabular_df_means, test_df, on='id')

X_pred = test_df_n.drop(['period', 'id', 'score'], axis=1).fillna(method='bfill')

res = model.predict_proba(X_pred)[:,1]
res.round(6)
test_df.SCORE = res.round(6)

## Export final result to file

In [ ]:
test_df.SCORE = res.round(6)
test_df.to_csv('HaidaTarasMonkaS_test.txt', sep=',', index=False)